In [1]:
#https://detectron2.readthedocs.io/
import detectron2
import logging
from detectron2.utils.logger import setup_logger
setup_logger()
logger = logging.getLogger('detectron2')

In [2]:
#Paths
from pathlib import Path
base_folder = Path('..')
data_folder = base_folder/'data'/'modanet'
train_imgs_folder = data_folder/'train'
train_annotations = data_folder/'ntrain.json'
val_imgs_folder = data_folder/'train'
val_annotations = data_folder/'nval.json'
#test_imgs_folder = data_folder/'CV_interim_images'

save_model_folder = base_folder/'ckpts'
load_model_folder = base_folder/'final_ckpts'

In [3]:
#register datasets. now they can be used as if they were native. remember to run fix_annotations.py to convert TIL2020 to proper COCO format
#to implement custom loaders, such as pickled, https://detectron2.readthedocs.io/modules/data.html?highlight=DatasetCatalog#detectron2.data.DatasetCatalog
from detectron2.data.datasets import register_coco_instances
register_coco_instances("moda_train", {}, train_annotations, train_imgs_folder)
register_coco_instances("moda_val", {}, val_annotations, val_imgs_folder)

In [4]:
#https://detectron2.readthedocs.io/modules/config.html
#btw, i added some custom config options for my checkpointer and pipeline
from detectron2 import model_zoo
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
#cfg.SEED = 42

#All the images in this dataset are 400x600... hey but 4 times the TIL data
cfg.DATASETS.TRAIN = ("moda_train",)
cfg.DATASETS.TEST = ("moda_val",)
cfg.OUTPUT_DIR = str(save_model_folder)

cfg.DATALOADER.NUM_WORKERS = 12
cfg.DATALOADER.SAMPLER_TRAIN = "TrainingSampler" #deals with class imbalance

cfg.SOLVER.IMS_PER_BATCH = 3 #batch_size
cfg.SOLVER.BASE_LR = 0.000375
cfg.SOLVER.GAMMA = 0.5 #halves the learning rate at each milestone
cfg.SOLVER.STEPS = (100800000,)#(20000,60000,100000,140000) # milestones in iterations
#^I played with these but realised it had little effect: so keep BASE_LR at 0.000125*IMS_PER_BATCH

#Pipeline augmentation settings (i implemented these)
cfg.INPUT.CROP.ENABLED = False
cfg.INPUT.ENABLE_MINOR_AUGMENTS = False

cfg.MODEL.ROI_HEADS.NUM_CLASSES = 13 #number of categories
cfg.PRINT_EVERY = 500
cfg.SAVE_EVERY = 2000 #when using ValCheckpointer, it saves only if val loss is the minimum so far

cfg.SOLVER.MAX_ITER = 1000000

logger.info(f"Loaded Config:\n{cfg.dump()}")

[06/16 17:08:48 detectron2]: Loaded Config:
CUDNN_BENCHMARK: false
DATALOADER:
  ASPECT_RATIO_GROUPING: true
  FILTER_EMPTY_ANNOTATIONS: true
  NUM_WORKERS: 12
  REPEAT_THRESHOLD: 0.0
  SAMPLER_TRAIN: TrainingSampler
DATASETS:
  PRECOMPUTED_PROPOSAL_TOPK_TEST: 1000
  PRECOMPUTED_PROPOSAL_TOPK_TRAIN: 2000
  PROPOSAL_FILES_TEST: []
  PROPOSAL_FILES_TRAIN: []
  TEST:
  - moda_val
  TRAIN:
  - moda_train
GLOBAL:
  HACK: 1.0
INPUT:
  CROP:
    ENABLED: false
    SIZE:
    - 0.9
    - 0.9
    TYPE: relative_range
  FORMAT: BGR
  MASK_FORMAT: polygon
  MAX_SIZE_TEST: 1333
  MAX_SIZE_TRAIN: 1333
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN:
  - 640
  - 672
  - 704
  - 736
  - 768
  - 800
  MIN_SIZE_TRAIN_SAMPLING: choice
MODEL:
  ANCHOR_GENERATOR:
    ANGLES:
    - - -90
      - 0
      - 90
    ASPECT_RATIOS:
    - - 0.5
      - 1.0
      - 2.0
    NAME: DefaultAnchorGenerator
    OFFSET: 0.0
    SIZES:
    - - 32
    - - 64
    - - 128
    - - 256
    - - 512
  BACKBONE:
    FREEZE_AT: 2
    NAME: 

In [5]:
from detectron2.modeling import build_model
from scripts.trainer import do_train,do_test
model = build_model(cfg)
logger.info(f"Created Model:\n{model}")

(norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
          )
        )
        (2): BottleneckBlock(
          (conv1): Conv2d(
            256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
          )
          (conv2): Conv2d(
            64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
          )
          (conv3): Conv2d(
            64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
          )
        )
      )
      (res3): Sequential(
        (0): BottleneckBlock(
          (shortcut): Conv2d(
            256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False
            (norm): FrozenBatchNorm2d(num_features=512, eps=1e-05)
          )
          (conv1): Conv2d(
            256, 128, kernel_size=(1, 1), stride=(2, 2), bias=False
            (no

In [6]:
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = str(load_model_folder/"ft-til_resnet101_rcnn_moda-65999-best_val.pth")
do_train(cfg,model,"ft-til_resnet101_rcnn_moda",resume=False)
#if this doesnt work correctly, check what model is set in the  file in /ckpts/last_checkpoint
#I uploaded the model file to the google drive

[06/16 17:09:03 detectron2]: Pipeline: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[06/16 17:09:06 d2.data.datasets.coco]: Loading ../data/modanet/ntrain.json takes 3.06 seconds.
[06/16 17:09:06 d2.data.datasets.coco]: Loaded 51208 images in COCO format from ../data/modanet/ntrain.json
[06/16 17:09:08 d2.data.build]: Removed 0 images with no usable annotations. 51208 images left.
[06/16 17:09:09 d2.data.build]: Distribution of instances among all 13 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|    bag     | 20444        |    belt    | 14056        |   boots    | 13720        |
|  footwear  | 75547        |   outer    | 24701        |   dress    | 14299        |
| sunglasses | 8604         |   pants    | 22723        |    top     | 35561        |
|   shorts   | 6923      

In [9]:
#if do_train() hasnt been called (it loads the model) and you want to load the model
if True:
    from detectron2.checkpoint import DetectionCheckpointer
    cfg.MODEL.WEIGHTS = str(load_model_folder/"ft-til_resnet101_rcnn_moda-65999-best_val.pth")
    DetectionCheckpointer(model, cfg.OUTPUT_DIR).resume_or_load(cfg.MODEL.WEIGHTS, resume=False)

do_test(cfg,model)

[06/16 16:24:25 d2.data.datasets.coco]: Loaded 1046 images in COCO format from modanet/nval.json
[06/16 16:24:25 d2.data.build]: Distribution of instances among all 13 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|    bag     | 432          |    belt    | 301          |   boots    | 239          |
|  footwear  | 1612         |   outer    | 507          |   dress    | 293          |
| sunglasses | 193          |   pants    | 450          |    top     | 729          |
|   shorts   | 152          |   skirt    | 282          |  headwear  | 113          |
| scarf/tie  | 107          |            |              |            |              |
|   total    | 5410         |            |              |            |              |
[06/16 16:24:25 d2.data.common]: Serializing 1046 elements to byte tensors and concatenating them all ...
[06/16 16:24:25 d2.data.co

In [5]:
from detectron2.engine import DefaultPredictor
cfg.MODEL.WEIGHTS = str(load_model_folder/"ft-til_resnet101_rcnn-17999-best_val.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.0 #max AP at cost of inference speed
predictor = DefaultPredictor(cfg) #check what exactly default predictor does, and if it affects anything badly.
#Are output boxes supposed to be rescaled? btw, the current pipeline resizes back to original dimensions

In [6]:
def boxes2xywh(boxes):
    return [[x1,y1,x2-x1,y2-y1] for x1,y1,x2,y2 in boxes.tensor.tolist()]

def convert_prediction(img_id,output):
    outs = []
    raw = output['instances']
    boxes = boxes2xywh(raw.pred_boxes)
    cats = raw.pred_classes.tolist()
    scores = raw.scores.tolist()

    for i in range(len(raw)):
        outs.append({
            'image_id':img_id,
            'category_id':cats[i],
            'bbox':boxes[i],
            'score':scores[i]
        })
    return outs


In [9]:
import json
from tqdm import tqdm
import numpy as np
import cv2

output = []
for im_path in tqdm(list(test_imgs_folder.glob('*'))):
    im = cv2.imread(im_path) #we are using cv2 here to be absolutely sure; this is what is in the documentation
    output += convert_prediction(int(im_path.stem),predictor(im))

with open('ans.json','w') as f:
    json.dump(output,f)

100%|██████████| 1000/1000 [02:44<00:00,  6.07it/s]


In [16]:
import cv2
from IPython.display import display
from PIL import Image
import numpy as np

from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
im = cv2.imread(train_imgs_folder/"69.jpg")
outputs = predictor(im)
v = Visualizer(im[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TEST[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
im_out = Image.fromarray(v.get_image()[:, :, ::-1]) #channels are reversed
#display(im_out)